In [1]:
from FinancialMachineLearning.features import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
os.chdir('..')
os.chdir('..')
os.getcwd()

'/Users/junghunlee/Desktop/Pycharm/FinancialMachineLearning'

In [3]:
data = pd.read_parquet('./Data/clean_IVE_fut_prices.parq')
data_price = data.iloc[:,:3]
data_volume = data.iloc[:,3:]

In [4]:
data_price = data_price.groupby(data_price.index).mean()
data_volume = data_volume.groupby(data_volume.index).sum()
data = pd.concat([data_price,data_volume], axis = 1)
data

,price,bid,ask,size,v,dv
dates,,,,,,
2009-09-28 09:30:00,50.750000,50.700000,50.79,738,738,37431.98
2009-09-28 09:31:32,50.750000,50.750000,50.76,100,100,5075.00
2009-09-28 09:31:33,50.750000,50.720000,50.75,100,100,5075.00
2009-09-28 09:31:50,50.750000,50.730000,50.76,300,300,15225.00
2009-09-28 09:31:51,50.750000,50.740000,50.76,300,300,15225.00
...,...,...,...,...,...,...
2018-10-05 15:59:57,116.200000,116.180000,116.20,200,200,23240.00
2018-10-05 15:59:59,116.197778,116.177778,116.19,9493,9493,1103077.60
2018-10-05 16:00:00,116.220000,116.200000,116.21,42884,42884,4983978.48


#### Tick rule

In [5]:
def BarSampling(df, column, threshold, tick = False):
    t = df[column]
    ts = 0
    idx = []
    if tick:
        for i, x in enumerate(t):
            ts += 1
            if ts >= threshold:
                idx.append(i)
                ts = 0
    else:
        for i, x in enumerate(t):
            ts += x
            if ts >= threshold:
                idx.append(i)
                ts = 0
    return df.iloc[idx].drop_duplicates()


In [6]:
dv = BarSampling(data, 'dv', 1000000)
dv

,price,bid,ask,size,v,dv
dates,,,,,,
2009-09-28 09:46:35,51.070000,51.055000,51.0725,6300,6300,321741.00
2009-09-28 09:54:35,51.130000,51.110000,51.1300,1500,1500,76695.00
2009-09-28 09:58:26,51.080000,51.068000,51.0800,1500,1500,76620.00
2009-09-28 10:02:52,51.255364,51.244545,51.2600,9200,9200,471521.90
2009-09-28 10:10:21,51.290000,51.280000,51.2900,5600,5600,287224.00
...,...,...,...,...,...,...
2018-10-05 15:58:47,116.160000,116.160000,116.1700,800,800,92928.00
2018-10-05 15:59:22,116.140000,116.140000,116.1500,1216,1216,141226.24
2018-10-05 15:59:56,116.200000,116.180000,116.2000,3800,3800,441560.00


In [8]:
from FinancialMachineLearning.features.microstructure import TickRule
aggressor = TickRule(dv['price'])

In [9]:
aggressor

dates
2009-09-28 09:46:35    1.0
2009-09-28 09:54:35    1.0
2009-09-28 09:58:26   -1.0
2009-09-28 10:02:52    1.0
2009-09-28 10:10:21    1.0
                      ... 
2018-10-05 15:58:47   -1.0
2018-10-05 15:59:22   -1.0
2018-10-05 15:59:56    1.0
2018-10-05 15:59:59   -1.0
2018-10-05 16:00:00    1.0
Length: 47936, dtype: float64

#### The Roll Model

In [10]:
from FinancialMachineLearning.features.microstructure import RollModel

spread, noise = RollModel(dv['price'])
spread, noise

(0.35672002824308546, -0.21981128335370995)

#### Range Volatility

In [ ]:
from FinancialMachineLearning.features.microstructure import RangeVolatility

vol = RangeVolatility()